# La typologie des variantes

In [7]:
import collatex
import json
import time

In [2]:
# On va importer les données lemmatisées
with open("Catullus/annotated/TEXT-O1.txt", "r") as input_text:
    text_1 = [line.replace("\n", "") for line in input_text.readlines()][1:]

# et 2
with open("Catullus/annotated/TEXT-G2.txt", "r") as input_text:
    text_2 = [line.replace("\n", "") for line in input_text.readlines()][1:]


# et 3
with open("Catullus/annotated/TEXT-Bodmer47-1.txt", "r") as input_text:
    text_3 = [line.replace("\n", "") for line in input_text.readlines()][1:]

In [3]:
def import_annotated_data(input_text:str):
    output_list = []
    for line in input_text:
        splits = line.split("\t")[:-2]
        output_list.append(splits)
    return output_list

def create_json_input_for_collatex(dict_of_text):
    output_dict = {"witnesses": []}
    for sigla, text in dict_of_text.items():
        witness_dict = {"id": sigla}
        tokens = []
        for token in text:
            form, lemma, pos, morph = token
            tokens.append({"t": form, "n":lemma, "lemma": lemma, "pos": pos, "morph": morph})
        witness_dict["tokens"] = tokens
        output_dict["witnesses"].append(witness_dict)
    return output_dict

In [11]:
text_1_as_list = import_annotated_data(text_1)
text_2_as_list = import_annotated_data(text_2)
text_3_as_list = import_annotated_data(text_3)
dict_of_text = {"O1": text_1_as_list, "G2": text_2_as_list, "Bodmer47": text_3_as_list}

json_input = create_json_input_for_collatex(dict_of_text)

In [12]:
resultat_json = collatex.collate(json_input, output='json', segmentation=False, near_match=True)
alignment_results_as_json = json.loads(resultat_json)

In [10]:
# Si on compare avec l'alignement sur les formes, on a des erreurs en moins.
result_table = collatex.collate(json_input, layout='vertical', segmentation=False)
print(result_table)

+------------+------------+--------------+
|     O1     |     G2     |   Bodmer47   |
+------------+------------+--------------+
|     -      |   fletus   |      -       |
+------------+------------+--------------+
|     -      |  passeris  |   Passeris   |
+------------+------------+--------------+
|     -      |   lesbie   |  appelatio   |
+------------+------------+--------------+
|   Passer   |   Passer   |    Passer    |
+------------+------------+--------------+
|  delicie   |  delicie   |   delitiae   |
+------------+------------+--------------+
|    mee     |    mee     |     meae     |
+------------+------------+--------------+
|   puelle   |   puelle   |   puellae    |
+------------+------------+--------------+
|    Qui     |    Qui     |     Qui      |
+------------+------------+--------------+
|    cum     |    cum     |     cum      |
+------------+------------+--------------+
|   ludere   |   ludere   |    ludere    |
+------------+------------+--------------+
|    quem  

### Essayons de produire une typologie des variantes

In [96]:
# Cette cellule charge les fonctions principales permettant d'analyser les variantes

def check_morph(locus):
    all_morphs = [witness['morph'] for witness in locus]
    print(f"Vérifions la morphologie: {all_morphs}")
    if all([morph == all_morphs[0] for morph in all_morphs[1:]]):
        print("La morphologie est identique.")
        return {'morph': True}
    else:
        print("Une différence morphologique semble apparaître: variante syntaxique ou grammaticale")
        return {'morph': False}

def check_lemmas(locus):
    all_lemmas = [witness['lemme'] for witness in locus]
    all_lemmas_as_string = " | ".join(all_lemmas)
    print(f"Vérifions les lemmes: {all_lemmas_as_string}")
    if all([lemma == all_lemmas[0] for lemma in all_lemmas[1:]]):
        print("Les lemmes sont identiques.")
        return {**check_morph(locus), **{"lemmas": True}}
    else:
        print("Les lemmes sont distincts. Variante lexicale")
        return {"lemmas": False, "morph":"UNK"}

def simplify_results(alignment_results_as_json):
    zipped = list(zip(alignment_results_as_json['table'][0], alignment_results_as_json['table'][1], alignment_results_as_json['table'][2]))

    output_data = list()
    for locus in zipped:
        interm_list = []
        for index, witness in enumerate(locus):
            if witness is not None:
                interm_dict = dict()
                interm_dict['témoin'] = witness[0]['_sigil']
                interm_dict['forme'] = witness[0]['t']
                interm_dict['lemme'] = witness[0]['n']
                interm_dict['pos'] = witness[0]['pos']
                interm_dict['morph'] = witness[0]['morph']
                interm_list.append(interm_dict)
            else:
                interm_dict = dict()
                interm_dict['témoin'] = alignment_results_as_json["witnesses"][index]
                interm_dict['forme'] = None
                interm_dict['lemme'] = None
                interm_dict['morph'] = None
                interm_dict['pos'] = None
                interm_list.append(interm_dict)
        output_data.append(interm_list)
    return output_data

def analyse_lieux_variants(collatex_output):
    results = simplify_results(collatex_output)
    for locus in results:
        print(f"Nouveau lieu variant.")
        print(f"Comparons les formes: {' | '.join([witness['forme'] if witness['forme'] != None else 'ø' for witness in locus])}")
        forme_base = locus[0]['forme']
        print(f"La forme base de la comparaison est: {forme_base}")
        if all([witness['forme'] == forme_base for witness in locus]):
            print("Toutes les formes sont identiques, il n'y a pas de lieu variant.")
        elif any([witness['forme'] == None for witness in locus]):
            all_forms = [witness['forme'] for witness in locus if witness['forme'] != None]
            all_forms_as_string = " | ".join(all_forms)
            print(f"On note une omission à cet endroit du texte. \nVérifions si les autres témoins concordent: {all_forms_as_string}")
            if all([form == all_forms[0] for form in all_forms[1:]]):
                print("Les autres témoins concordent. Omission")
            else:
                print("Les autres témoins discordent dans leur forme")
                locus = [witness for witness in locus if witness['forme'] != None]
                check_lemma = check_lemmas(locus)
                if check_lemma['morph'] == True and check_lemma['lemmas'] == True:
                    print("Variante graphique")
        else:
            print("Les témoins discordent dans leur forme.")
            check_lemma = check_lemmas(locus)
            if check_lemma['morph'] == True and check_lemma['lemmas'] == True:
                print("Variante graphique")
            
    
        print("\n")

In [97]:
# On prend la sortie de collatex et on analyse chaque unité d'alignement à la suite
analyse_lieux_variants(alignment_results_as_json)

            

Nouveau lieu variant.
Comparons les formes: ø | fletus | ø
La forme base de la comparaison est: None
On note une omission à cet endroit du texte. 
Vérifions si les autres témoins concordent: fletus
Les autres témoins concordent. Omission


Nouveau lieu variant.
Comparons les formes: ø | passeris | Passeris
La forme base de la comparaison est: None
On note une omission à cet endroit du texte. 
Vérifions si les autres témoins concordent: passeris | Passeris
Les autres témoins discordent dans leur forme
Vérifions les lemmes: patior | patior
Les lemmes sont identiques.
Vérifions la morphologie: ['Numb=Sing|Person=2', 'Numb=Sing|Person=2']
La morphologie est identique.
Variante graphique


Nouveau lieu variant.
Comparons les formes: ø | lesbie | appelatio
La forme base de la comparaison est: None
On note une omission à cet endroit du texte. 
Vérifions si les autres témoins concordent: lesbie | appelatio
Les autres témoins discordent dans leur forme
Vérifions les lemmes: lesbie | appelatio
L

Comme on le voit, le processus est très sensible à la qualité de l'annotation et de la lemmatisation, qui est lui-même dépendant de la variabilité graphique des témoins. La phase d'annotation lexico-grammaticale est donc fondamentale et les modèles d'annotation doivent être le plus précis possible.